# Recommender System

Books

April 2019

Based on recommender system architecture [here](https://github.com/cipher813/recommender_system) and Book Crossing dataset [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).

In [1]:
from bm_util import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH = "/tmp/data/"

In [3]:
fp = PATH + "BX-CSV-Dump.zip"
url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
fp = download_url_to_filepath(fp, url)
unzip_file(fp,PATH)
os.listdir(PATH)

['BX-Book-Ratings.csv',
 'BX-Books.csv',
 'BX-CSV-Dump.zip',
 'BX-Users.csv',
 'wine.data']

In [4]:
def read_BX_csv(filepath,encoding):
    df = pd.read_csv(filepath,delimiter=';',error_bad_lines=False,encoding=encoding)
    return df

def print_info(df):
    print(f"Shape:\n{df.shape}")
    print(f"Info:\n{df.info()}")
    print(f"Unique:\n{df.nunique()}")

In [5]:
br = read_BX_csv(PATH + "BX-Book-Ratings.csv",'unicode_escape')
bu = read_BX_csv(PATH + "BX-Users.csv",'unicode_escape')
bk = read_BX_csv(PATH + "BX-Books.csv",'latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [6]:
for df in [("Ratings",br), ("Users",bu), ("Info",bk)]:
    print(f"\n{df[0]}\n")
    print_info(df[1])


Ratings

Shape:
(1149780, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
Info:
None
Unique:
User-ID        105283
ISBN           340556
Book-Rating        11
dtype: int64

Users

Shape:
(278858, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User-ID     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
Info:
None
Unique:
User-ID     278858
Location     57339
Age            165
dtype: int64

Info

Shape:
(271360, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
ISBN                   271360 non-null object
Book-Title         

In [7]:
df1 = br.merge(bu,how='left',on='User-ID')
df = df1.merge(bk,how='left',on='ISBN')
print(df.shape)
df.head()

(1149780, 12)


,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,"seattle, washington, usa",NaN,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,"h, new south wales, australia",16.0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,"rijeka, n/a, croatia",16.0,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


### Prepare Sparse Matrix

In [10]:
print(br.shape)
br.head()

(1149780, 3)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [19]:
br1 = pd.DataFrame(br.groupby("User-ID")["ISBN"].count()).sort_values("ISBN",ascending=False)
user_list = br1[br1['ISBN']>10].index
user_list

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            182212, 182423, 177015, 130806, 165267, 128974, 117543, 141954,
             24697, 150894],
           dtype='int64', name='User-ID', length=12053)

In [22]:
br2 = pd.DataFrame(br.groupby("ISBN")["User-ID"].count()).sort_values("User-ID",ascending=False)
book_list = br2[br2["User-ID"]>10].index
book_list

Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '1400047838', '0440222788', '0375506764', '1551665891', '0618093974',
       '0670823171', '0440229103', '0140187758', '0752848229', '0449912639'],
      dtype='object', name='ISBN', length=16162)

In [8]:
pivot = br.pivot(index="ISBN",columns="User-ID",values="Book-Rating")

KeyboardInterrupt: 